In this notebook we prepare Explanation dataset


*   first use sanity test for creating small dataset
*   second  preprocess dataset
*   third  use sbert to create embedding for nodes and edges



Install torch geometric

import necessary libraries

In [1]:
import re
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch_geometric.data.data import Data
import torch
from torch import nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import DataLoader
import numpy as np
import glob



/home/ahmadi/sadaf/GraphNeighborLM/sadafenv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


mount to  my drive

Explanation graph dataset

In [31]:
path = '/home/ahmadi/sadaf/GraphNeighborLM/G-retriever/datasets/Explanation_graph'
original_dataset = pd.read_csv(f'{path}/train_dev.tsv', sep='\t')

original_dataset.head()

,arg1,arg2,label,graph
0,Cannabis should be legal.,It's not a bad thing to make marijuana more av...,support,(cannabis; synonym of; marijuana)(legal; cause...
1,Women should not be in combat.,Women and men have the same rights.,counter,(women and men; is a; citizens)(citizens; caus...
2,People will use marijuana independent of its l...,People use marijuana everywhere now.,support,(marijuana; receives action; popular)(popular;...
3,women should not partake in war,the armed forces are more open to recruiting w...,counter,(armed forces; desires; nurses and helpers)(nu...
4,Marijuana should not be legalized.,Marijuana is dangerous for society.,support,(marijuana; is a; recreational drug)(recreatio...


In [32]:
original_dataset.shape

(2766, 4)

In [33]:
original_dataset.graph.iloc[0]

'(cannabis; synonym of; marijuana)(legal; causes; more available)(marijuana; capable of; good thing)(good thing; desires; legal)'

Saniti test

In [34]:
seed = 0
percent_data = 0.05
dataset_sample = original_dataset
X_train, dataset = train_test_split(dataset_sample, test_size = percent_data, random_state = seed)

In [35]:
dataset.shape

(139, 4)

In [36]:
dataset

,arg1,arg2,label,graph
1436,Organ acquisition in the market makes it easie...,Sale of organ in the market makes it easy to g...,support,(organ acquisition; capable of; more organs av...
817,the three strikes law is not fair.,The three strikes law keeps people safe.,counter,(three strikes law; capable of; keeps people s...
2708,Cloning is inherently decreasing quality,Getting your original out of the copier and pu...,support,(cloning; synonym of; copy)(copy; capable of; ...
2199,Three-strike laws help reduce crime rates.,To say that three-strike laws reduce crime rat...,counter,(three-strike laws; not capable of; assist)(as...
1074,"Telemarketing is fast, safe and reliable.",Telemarketing helps business advertise and get...,support,(telemarketing; capable of; secure)(secure; ha...
...,...,...,...,...
728,the military should not be based upon profit.,The military based upon profit would be a disa...,support,(military; made of; independent)(independent; ...
346,missionary works helps people.,Missionary works helps those who are in need.,support,(missionary works; has subevent; charity work)...
2229,Since payday loans aren't held to the same sta...,Poor people have money emergencies.,counter,(solve emergencies; is a; helpful)(poor people...
1759,"If everyone has to use a public defender, it m...","Anyone who can afford it, has the right to any...",counter,(can afford; used for; any defender)(any defen...


In [37]:
dataset.to_csv("/home/ahmadi/sadaf/GraphNeighborLM/G-retriever/datasets/Explanation_graph/sample_train_dev.tsv", index=False)
dataset=pd.read_csv("/home/ahmadi/sadaf/GraphNeighborLM/G-retriever/datasets/Explanation_graph/sample_train_dev.tsv")

In [38]:
dataset

,arg1,arg2,label,graph
0,Organ acquisition in the market makes it easie...,Sale of organ in the market makes it easy to g...,support,(organ acquisition; capable of; more organs av...
1,the three strikes law is not fair.,The three strikes law keeps people safe.,counter,(three strikes law; capable of; keeps people s...
2,Cloning is inherently decreasing quality,Getting your original out of the copier and pu...,support,(cloning; synonym of; copy)(copy; capable of; ...
3,Three-strike laws help reduce crime rates.,To say that three-strike laws reduce crime rat...,counter,(three-strike laws; not capable of; assist)(as...
4,"Telemarketing is fast, safe and reliable.",Telemarketing helps business advertise and get...,support,(telemarketing; capable of; secure)(secure; ha...
...,...,...,...,...
134,the military should not be based upon profit.,The military based upon profit would be a disa...,support,(military; made of; independent)(independent; ...
135,missionary works helps people.,Missionary works helps those who are in need.,support,(missionary works; has subevent; charity work)...
136,Since payday loans aren't held to the same sta...,Poor people have money emergencies.,counter,(solve emergencies; is a; helpful)(poor people...
137,"If everyone has to use a public defender, it m...","Anyone who can afford it, has the right to any...",counter,(can afford; used for; any defender)(any defen...


Train test split

In [39]:
indices = np.arange(10)
train_indices, temp_data = train_test_split(indices, test_size=0.4, random_state=42)
train_indices

array([7, 2, 9, 4, 3, 6])

save indices of train , val, test in txt file

In [40]:
def generate_split(num_nodes, path):

    # Split the dataset into train, val, and test sets
    indices = np.arange(num_nodes)

    # Make a small training set 60%, a validation set 20%, and a test set 20%
    train_indices, temp_data = train_test_split(indices, test_size=0.4, random_state=42)
    val_indices, test_indices = train_test_split(temp_data, test_size=0.5, random_state=42)
    print("# train samples: ", len(train_indices))
    print("# val samples: ", len(val_indices))
    print("# test samples: ", len(test_indices))

    # Create a folder for the split
    os.makedirs(path, exist_ok=True)

    # Save the indices to separate files
    with open(f'{path}/train_indices.txt', 'w') as file:
        file.write('\n'.join(map(str, train_indices)))

    with open(f'{path}/val_indices.txt', 'w') as file:
        file.write('\n'.join(map(str, val_indices)))

    with open(f'{path}/test_indices.txt', 'w') as file:
        file.write('\n'.join(map(str, test_indices)))

Extracting nodes and edges from graph in dataset

In [41]:
def textualize_graph(graph):
    triplets = re.findall(r'\((.*?)\)', graph)
    nodes = {}
    edges = []
    for tri in triplets:
        src, edeg_attr, dst = tri.split(';')
        src = src.lower().strip()
        dst = dst.lower().strip()
        if src not in nodes:
            nodes[src] = len(nodes)
        if dst not in nodes:
            nodes[dst] = len(nodes)
        edges.append({'src': nodes[src], 'edge_attr': edeg_attr.lower().strip(), 'dst': nodes[dst], })

    nodes = pd.DataFrame(nodes.items(), columns=['node_attr', 'node_id'])
    edges = pd.DataFrame(edges)
    return nodes, edges

In [42]:
textualize_graph('(al;fa;vo)')

(  node_attr  node_id
 0        al        0
 1        vo        1,
    src edge_attr  dst
 0    0        fa    1)

Save nodes and edges in csv

In [43]:
def step_one():
    # generate textual graphs
    os.makedirs(f'{path}/nodes', exist_ok=True)
    os.makedirs(f'{path}/edges', exist_ok=True)

    for i, row in tqdm(dataset.iterrows(), total=len(dataset)):
        nodes, edges = textualize_graph(row['graph'])
        nodes.to_csv(f'{path}/nodes/{i}.csv', index=False, columns=['node_id', 'node_attr'])
        edges.to_csv(f'{path}/edges/{i}.csv', index=False, columns=['src', 'edge_attr', 'dst'])

In [ ]:
#the nodes of the first row in dataset
pd.read_csv("//content//drive//MyDrive//G-Retriever-git//dataset//expla_graphs//nodes//1.csv")

,node_id,node_attr
0,0,three strikes law
1,1,keeps people safe
2,2,just
3,3,fair


In [ ]:
#the edges of the first row in dataset
pd.read_csv("//content//drive//MyDrive//G-Retriever-git//dataset//expla_graphs//edges//1.csv")

,src,edge_attr,dst
0,0,capable of,1
1,1,is a,2
2,2,synonym of,3


In [44]:

pretrained_repo = 'sentence-transformers/all-roberta-large-v1' # It maps sentences & paragraphs to a 1024 dimensional dense vector space
batch_size = 8  # Adjust the batch size as needed
model_name = 'sbert'


In [19]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, input_ids=None, attention_mask=None):
        super().__init__()
        self.data = {
            "input_ids": input_ids,
            "att_mask": attention_mask,
        }

    def __len__(self):
        return self.data["input_ids"].size(0)

    def __getitem__(self, index):
        if isinstance(index, torch.Tensor):
            index = index.item()
        batch_data = dict()
        for key in self.data.keys():
            if self.data[key] is not None:
                batch_data[key] = self.data[key][index]
        return batch_data


this function create embedding and the original code is on huggingface


In [20]:
class Sentence_Transformer(nn.Module):

    def __init__(self, pretrained_repo):
        super(Sentence_Transformer, self).__init__()
        print(f"inherit model weights from {pretrained_repo}")
        self.bert_model = AutoModel.from_pretrained(pretrained_repo)

    def mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0]  # First element of model_output contains all token embeddings
        data_type = token_embeddings.dtype
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).to(data_type)
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    def forward(self, input_ids, att_mask):
        bert_out = self.bert_model(input_ids=input_ids, attention_mask=att_mask)
        sentence_embeddings = self.mean_pooling(bert_out, att_mask)

        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings


original code on hugging face is:

In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch
# import torch.nn.functional as F

# #Mean Pooling - Take attention mask into account for correct averaging
# def mean_pooling(model_output, attention_mask):
#     token_embeddings = model_output[0] #First element of model_output contains all token embeddings
#     input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
#     return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# # Sentences we want sentence embeddings for
# sentences = ['This is an example sentence', 'Each sentence is converted']

# # Load model from HuggingFace Hub
# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-roberta-large-v1')
# model = AutoModel.from_pretrained('sentence-transformers/all-roberta-large-v1')

# # Tokenize sentences
# encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# # Compute token embeddings
# with torch.no_grad():
#     model_output = model(**encoded_input)

# # Perform pooling
# sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# # Normalize embeddings
# sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

# print("Sentence embeddings:")
# print(sentence_embeddings)


In [21]:
def load_sbert():

    model = Sentence_Transformer(pretrained_repo)
    tokenizer = AutoTokenizer.from_pretrained(pretrained_repo)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    model.eval()
    return model, tokenizer, device

In [45]:
load_sbert()

inherit model weights from sentence-transformers/all-roberta-large-v1


(Sentence_Transformer(
   (bert_model): RobertaModel(
     (embeddings): RobertaEmbeddings(
       (word_embeddings): Embedding(50265, 1024, padding_idx=1)
       (position_embeddings): Embedding(514, 1024, padding_idx=1)
       (token_type_embeddings): Embedding(1, 1024)
       (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
       (dropout): Dropout(p=0.1, inplace=False)
     )
     (encoder): RobertaEncoder(
       (layer): ModuleList(
         (0-23): 24 x RobertaLayer(
           (attention): RobertaAttention(
             (self): RobertaSelfAttention(
               (query): Linear(in_features=1024, out_features=1024, bias=True)
               (key): Linear(in_features=1024, out_features=1024, bias=True)
               (value): Linear(in_features=1024, out_features=1024, bias=True)
               (dropout): Dropout(p=0.1, inplace=False)
             )
             (output): RobertaSelfOutput(
               (dense): Linear(in_features=1024, out_features=1024, 

In [46]:
def sber_text2embedding(model, tokenizer, device, text):
    try:
        encoding = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        dataset = Dataset(input_ids=encoding.input_ids, attention_mask=encoding.attention_mask)

        # DataLoader
        dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

        # Placeholder for storing the embeddings
        all_embeddings = []

        # Iterate through batches
        with torch.no_grad():

            for batch in dataloader:
                # Move batch to the appropriate device
                batch = {key: value.to(device) for key, value in batch.items()}

                # Forward pass
                embeddings = model(input_ids=batch["input_ids"], att_mask=batch["att_mask"])

                # Append the embeddings to the list
                all_embeddings.append(embeddings)

        # Concatenate the embeddings from all batches
        all_embeddings = torch.cat(all_embeddings, dim=0).cpu()

    except:
        return torch.zeros((0, 1024))

    return all_embeddings



In [47]:
load_model = {
    'sbert': load_sbert
}



load_text2embedding = {
    'sbert': sber_text2embedding
}

In [49]:
def step_two():

    def _encode_graph():
        print('Encoding graphs...')
        os.makedirs(f'{path}/graphs', exist_ok=True)
        # Set paths to nodes and edges directories
        nodes_path = f'{path}/nodes'
        edges_path = f'{path}/edges'

        # Get all CSV files in the directories
        node_files = glob.glob(f'{nodes_path}/*.csv')
        edge_files = glob.glob(f'{edges_path}/*.csv')

        # Ensure lists are sorted (if they need to correspond to each other)
        node_files.sort()
        edge_files.sort()

        for i in tqdm(range(len(dataset))):
            nodes = pd.read_csv(node_files[i])
            edges = pd.read_csv(edge_files[i])

            x = text2embedding(model, tokenizer, device, nodes.node_attr.tolist())
            e = text2embedding(model, tokenizer, device, edges.edge_attr.tolist())
            edge_index = torch.LongTensor([edges.src, edges.dst])
            data = Data(x=x, edge_index=edge_index, edge_attr=e, num_nodes=len(nodes))
            torch.save(data, f'{path}/graphs/{i}.pt')

    model, tokenizer, device = load_model[model_name]()
    text2embedding = load_text2embedding[model_name]

    _encode_graph()


if __name__ == '__main__':
    step_one()
    step_two()
    generate_split(len(dataset), f'{path}/split')

  0%|          | 0/139 [00:00<?, ?it/s]

100%|██████████| 139/139 [00:00<00:00, 276.18it/s]


inherit model weights from sentence-transformers/all-roberta-large-v1
Encoding graphs...


100%|██████████| 139/139 [00:17<00:00,  8.16it/s]

# train samples:  83
# val samples:  28
# test samples:  28




```
# This is formatted as code
```

content of graph.pt

In [ ]:
# Define the path to the .pt file
file_path = f'{path}/graphs/{10}.pt'

# Load the data
data = torch.load(file_path)

# Check the type of the loaded data
print(f'Type of the loaded data: {type(data)}')

# Print the content of the loaded data
print(data)

# If data is an instance of torch_geometric.data.Data, you can access its attributes
if isinstance(data, Data):
    print("Node Features (x):", data.x)
    print("Edge Index (edge_index):", data.edge_index)
    print("Edge Attributes (edge_attr):", data.edge_attr)
    print("Number of Nodes (num_nodes):", data.num_nodes)
    print("Additional attributes:", data.keys)
else:
    print("The loaded data is not a PyTorch Geometric Data object.")


Type of the loaded data: <class 'torch_geometric.data.data.Data'>
Data(x=[5, 1024], edge_index=[2, 4], edge_attr=[4, 1024], num_nodes=5)
Node Features (x): tensor([[ 0.0116, -0.0097,  0.0214,  ..., -0.0062,  0.0419,  0.0353],
        [-0.0100, -0.0138,  0.0244,  ...,  0.0237,  0.0151,  0.0188],
        [-0.0223, -0.0328,  0.0352,  ...,  0.0150,  0.0384, -0.0085],
        [ 0.0091, -0.0231,  0.0435,  ..., -0.0063, -0.0028,  0.0418],
        [-0.0078, -0.0021,  0.0062,  ..., -0.0241,  0.0036, -0.0154]])
Edge Index (edge_index): tensor([[0, 1, 0, 3],
        [1, 2, 3, 4]])
Edge Attributes (edge_attr): tensor([[-0.0199,  0.0516,  0.0226,  ..., -0.0219, -0.0137, -0.0084],
        [-0.0814,  0.0128,  0.0008,  ...,  0.0169, -0.0142, -0.0408],
        [-0.0063,  0.0141,  0.0474,  ...,  0.0107, -0.0122, -0.0182],
        [-0.0528,  0.0143, -0.0078,  ...,  0.0071,  0.0013,  0.0375]])
Number of Nodes (num_nodes): 5
Additional attributes: <bound method BaseData.keys of Data(x=[5, 1024], edge_index

<ipython-input-30-597741838451>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [ ]:
data

Data(x=[8, 1024], edge_index=[2, 7], edge_attr=[7, 1024], num_nodes=8)

In [ ]:
dataset

,arg1,arg2,label,graph
0,Organ acquisition in the market makes it easie...,Sale of organ in the market makes it easy to g...,support,(organ acquisition; capable of; more organs av...
1,the three strikes law is not fair.,The three strikes law keeps people safe.,counter,(three strikes law; capable of; keeps people s...
2,Cloning is inherently decreasing quality,Getting your original out of the copier and pu...,support,(cloning; synonym of; copy)(copy; capable of; ...
3,Three-strike laws help reduce crime rates.,To say that three-strike laws reduce crime rat...,counter,(three-strike laws; not capable of; assist)(as...
4,"Telemarketing is fast, safe and reliable.",Telemarketing helps business advertise and get...,support,(telemarketing; capable of; secure)(secure; ha...
...,...,...,...,...
272,People have a choice as to whether or not they...,Wars have been fought over people's spiritual ...,support,(people; capable of; freedom of religion)(pray...
273,there can be a reductions in moral codes and v...,there can also be an increase in ethics and va...,counter,(increase; has context; moral codes)(increase;...
274,The surgery should not be banned,The surgery can harm the patient,counter,(surgery; capable of; harm)(harm; has context;...
275,Suicide is morally wrong ad it is a criminal o...,Assisted suicide helps victims get ou of their...,counter,(assisted suicide; not part of; criminal offen...


In [50]:
import json
import pandas as pd
import torch
from torch.utils.data import Dataset


PATH = '/home/ahmadi/sadaf/GraphNeighborLM/G-retriever/datasets/Explanation_graph'


class ExplaGraphsDataset(Dataset):
    def __init__(self):
        super().__init__()

        self.text = pd.read_csv(f'{PATH}/sample_train_dev.tsv', sep=',')
        self.prompt = 'Question: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of \'support\' or \'counter\'.\n\nAnswer:'
        self.graph = None
        self.graph_type = 'Explanation Graph'

    def __len__(self):
        """Return the len of the dataset."""
        return len(self.text)

    def __getitem__(self, index):

        text = self.text.iloc[index]
        graph = torch.load(f'{PATH}/graphs/{index}.pt')
        question = f'Argument 1: {text.arg1}\nArgument 2: {text.arg2}\n{self.prompt}'
        nodes = pd.read_csv(f'{PATH}/nodes/{index}.csv')
        edges = pd.read_csv(f'{PATH}/edges/{index}.csv')
        desc = nodes.to_csv(index=False)+'\n'+edges.to_csv(index=False)

        return {
            'id': index,
            'label': text['label'],
            'desc': desc,
            'graph': graph,
            'question': question,
        }

    def get_idx_split(self):

        # Load the saved indices
        with open(f'{PATH}/split/train_indices.txt', 'r') as file:
            train_indices = [int(line.strip()) for line in file]

        with open(f'{PATH}/split/val_indices.txt', 'r') as file:
            val_indices = [int(line.strip()) for line in file]

        with open(f'{PATH}/split/test_indices.txt', 'r') as file:
            test_indices = [int(line.strip()) for line in file]

        return {'train': train_indices, 'val': val_indices, 'test': test_indices}




In [51]:
nodes = pd.read_csv(f'{PATH}/nodes/{1}.csv')
edges = pd.read_csv(f'{PATH}/edges/{1}.csv')
desc = nodes.to_csv(index=False)+'\n'+edges.to_csv(index=False)
desc

'node_id,node_attr\n0,three strikes law\n1,keeps people safe\n2,just\n3,fair\n\nsrc,edge_attr,dst\n0,capable of,1\n1,is a,2\n2,synonym of,3\n'

In [52]:
print(desc)

node_id,node_attr
0,three strikes law
1,keeps people safe
2,just
3,fair

src,edge_attr,dst
0,capable of,1
1,is a,2
2,synonym of,3



In [53]:
if __name__ == '__main__':
    dataset = ExplaGraphsDataset()

    print(dataset.prompt)

    data = dataset[1]
    for k, v in data.items():
        print(f'{k}: {v}')

    split_ids = dataset.get_idx_split()
    for k, v in split_ids.items():
        print(f'# {k}: {len(v)}')


Question: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of 'support' or 'counter'.

Answer:
id: 1
label: counter
desc: node_id,node_attr
0,three strikes law
1,keeps people safe
2,just
3,fair

src,edge_attr,dst
0,capable of,1
1,is a,2
2,synonym of,3

graph: Data(x=[4, 1024], edge_index=[2, 3], edge_attr=[3, 1024], num_nodes=4)
question: Argument 1: the three strikes law is not fair.
Argument 2: The three strikes law keeps people safe.
Question: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of 'support' or 'counter'.

Answer:
# train: 83
# val: 28
# test: 28


In [ ]:
args= parse_args_llama()
args

Namespace(model_name='llm', project='project_g_retriever', seed=0, dataset='expla_graphs', lr=1e-05, wd=0.05, patience=2, batch_size=1, grad_steps=2, num_epochs=2, warmup_epochs=1, eval_batch_size=16, llm_model_name='7b', llm_model_path='', llm_frozen='False', llm_num_virtual_tokens=10, output_dir='output', max_txt_len=128, max_new_tokens=32, gnn_model_name='gt', gnn_num_layers=4, gnn_in_dim=1024, gnn_hidden_dim=1024, gnn_num_heads=4, gnn_dropout=0.0)

In [ ]:
model = load_model[args.model_name](graph_type=dataset.graph_type, args=args, init_prompt=dataset.prompt)
model

Loading LLAMA


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [54]:
# from src.model.llm import LLM
# from src.model.pt_llm import PromptTuningLLM
# from src.model.graph_llm import GraphLLM


load_model = {
    #"llm": LLM,
    # "inference_llm": LLM,
    # "pt_llm": PromptTuningLLM,
    "graph_llm": GraphLLM,
}

# Replace the following with the model paths
llama_model_path = {
    "7b": "edumunozsala/llama-2-7b-int4-python-code-20k",
    #"7b": "meta-llama/Llama-2-7b-hf",
    # "7b_chat": "meta-llama/Llama-2-7b-chat-hf",
    # "13b": "meta-llama/Llama-2-13b-hf",
    # "13b_chat": "meta-llama/Llama-2-13b-chat-hf",
}

In [3]:
import os
#import wandb
import gc
from tqdm import tqdm
import torch
import json
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_

# from src.model import load_model, llama_model_path
# from src.dataset import load_dataset
# from src.utils.evaluate import eval_funcs
# from src.config import parse_args_llama
# from src.utils.ckpt import _save_checkpoint, _reload_best_model
# from src.utils.collate import collate_fn
# from src.utils.seed import seed_everything
# from src.utils.lr_schedule import adjust_learning_rate

In [4]:
import random, os
import numpy as np
import torch

def seed_everything(seed: int):
    """
    Set a fixed random seed for reproducibility across multiple libraries and environments.

    This function ensures that experiments in machine learning and data science are reproducible
    by setting the same random seed for various sources of randomness, including Python's `random` module,
    NumPy, and PyTorch (for both CPU and GPU). It also configures PyTorch's CuDNN backend to enforce
    deterministic behavior.

    Args:
        seed (int): The seed value to set for all random number generators.

    Libraries Affected:
        - `random`: Sets the random seed for Python's built-in random module.
        - `os`: Sets the `PYTHONHASHSEED` environment variable to ensure deterministic hashing in Python.
        - `numpy`: Sets the random seed for NumPy.
        - `torch`: Sets the seed for PyTorch's random number generation on both CPU and GPU.

    PyTorch-Specific Settings:
        - `torch.backends.cudnn.deterministic = True`:
            Forces PyTorch to use deterministic algorithms for CuDNN operations.
        - `torch.backends.cudnn.benchmark = True`:
            Enables dynamic algorithm optimization in CuDNN for faster execution in some cases.

    Example:
        >>> seed_everything(42)
        >>> random.randint(0, 10)  # Always generates the same number
        >>> np.random.rand(3)  # Produces the same array for a fixed seed
        >>> torch.randn(3, 3)  # Produces the same tensor for a fixed seed

    Notes:
        - While `torch.backends.cudnn.deterministic = True` ensures reproducibility, it may slow down computations.
        - `torch.backends.cudnn.benchmark = True` may introduce slight variability in speed for different input sizes.

    """
    random.seed(seed) #for random library
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed) #for numpy
    torch.manual_seed(seed) #for cpu
    torch.cuda.manual_seed(seed)#for gpu
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [5]:
from torch_geometric.data import Batch


def collate_fn(original_batch):
    """

    Custom collate function for batching data in a PyTorch DataLoader.

    This function processes a batch of individual samples (dictionaries) from the dataset and combines them into a
    single batch dictionary. If the data contains graphs (PyTorch Geometric `Data` objects), they are batched
    into a single graph using `Batch.from_data_list`.

    Args:
        original_batch (list of dict): A batch of samples from the dataset. Each sample is a dictionary where keys
                                        represent feature names (e.g., "question", "labels", "desc", "graph"), and values
                                        are the corresponding data for each sample.

    Returns:
        dict: A dictionary containing batched data for all keys:
              - For non-graph data, values are lists of the corresponding data from each sample.
              - For graph data (key: "graph"), a single `Batch` object from PyTorch Geometric is returned.

              Example output:
              {
                  "input_ids": [[101, 200, 300], [102, 201, 301]],
                  "labels": [1, 0],
                  "graph": Batch(...)  # Batched PyTorch Geometric graph object
              }

    Notes:
        - If the dataset contains a "graph" key, it is expected to be a PyTorch Geometric `Data` object.
        - Non-graph features are combined into lists for easier downstream processing.

    """
    batch = {}
    for k in original_batch[0].keys():
        batch[k] = [d[k] for d in original_batch]
    if 'graph' in batch:
        batch['graph'] = Batch.from_data_list(batch['graph'])
    return batch

In [6]:
import json
import pandas as pd
import re
import string


# def get_accuracy_gqa(path):
#     df = pd.read_json(path, lines=True)
#     # compute accuracy
#     correct = 0
#     for pred, label in zip(df["pred"], df["label"]):
#         if label in pred:
#             correct += 1
#     return correct / len(df)


def get_accuracy_expla_graphs(path):
    df = pd.read_json(path, lines=True)
    # compute accuracy
    correct = 0
    for pred, label in zip(df["pred"], df["label"]):
        matches = re.findall(r"support|Support|Counter|counter", pred.strip())
        if len(matches) > 0 and matches[0].lower() == label:
            correct += 1

    return correct / len(df)


def normalize(s: str) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""
    s = s.lower()
    exclude = set(string.punctuation)
    s = "".join(char for char in s if char not in exclude)
    s = re.sub(r"\b(a|an|the)\b", " ", s)
    # remove <pad> token:
    s = re.sub(r"\b(<pad>)\b", " ", s)
    s = " ".join(s.split())
    return s


def match(s1: str, s2: str) -> bool:
    s1 = normalize(s1)
    s2 = normalize(s2)
    return s2 in s1


def eval_f1(prediction, answer):
    if len(prediction) == 0:
        return 0, 0, 0
    matched = 0
    prediction_str = " ".join(prediction)
    for a in answer:
        if match(prediction_str, a):
            matched += 1
    precision = matched / len(prediction)
    recall = matched / len(answer)
    if precision + recall == 0:
        return 0, precision, recall
    else:
        return 2 * precision * recall / (precision + recall), precision, recall


def eval_acc(prediction, answer):
    matched = 0.0
    for a in answer:
        if match(prediction, a):
            matched += 1
    return matched / len(answer)


def eval_hit(prediction, answer):
    for a in answer:
        if match(prediction, a):
            return 1
    return 0


# def get_accuracy_webqsp(path):
#     df = pd.read_json(path, lines=True)

#     # Load results
#     acc_list = []
#     hit_list = []
#     f1_list = []
#     precission_list = []
#     recall_list = []

#     for prediction, answer in zip(df.pred.tolist(), df.label.tolist()):

#         prediction = prediction.replace("|", "\n")
#         answer = answer.split("|")

#         prediction = prediction.split("\n")
#         f1_score, precision_score, recall_score = eval_f1(prediction, answer)
#         f1_list.append(f1_score)
#         precission_list.append(precision_score)
#         recall_list.append(recall_score)
#         prediction_str = " ".join(prediction)
#         acc = eval_acc(prediction_str, answer)
#         hit = eval_hit(prediction_str, answer)
#         acc_list.append(acc)
#         hit_list.append(hit)

#     acc = sum(acc_list) * 100 / len(acc_list)
#     hit = sum(hit_list) * 100 / len(hit_list)
#     f1 = sum(f1_list) * 100 / len(f1_list)
#     pre = sum(precission_list) * 100 / len(precission_list)
#     recall = sum(recall_list) * 100 / len(recall_list)

#     print(f"Accuracy: {acc:.4f}")
#     print(f"Hit: {hit:.4f}")
#     print(f"Precision: {pre:.4f}")
#     print(f"Recall: {recall:.4f}")
#     print(f"F1: {f1:.4f}")

#     return hit


eval_funcs = {
    "expla_graphs": get_accuracy_expla_graphs,
    # "scene_graphs": get_accuracy_gqa,
    # "scene_graphs_baseline": get_accuracy_gqa,
    # "webqsp": get_accuracy_webqsp,
    # "webqsp_baseline": get_accuracy_webqsp,
}

In [6]:
import os
import torch


def print_trainable_params(model):
    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()

        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    return trainable_params, all_param


def _save_checkpoint(model, optimizer, cur_epoch, args, is_best=False):
    """
    Save the checkpoint at the current epoch.
    """
    os.makedirs(f'{args.output_dir}/{args.dataset}', exist_ok=True)

    param_grad_dic = {
        k: v.requires_grad for (k, v) in model.named_parameters()
    }
    state_dict = model.state_dict() #learnable parameter
    for k in list(state_dict.keys()):
        if k in param_grad_dic.keys() and not param_grad_dic[k]:
            # delete parameters that do not require gradient
            del state_dict[k]
    save_obj = {
        "model": state_dict,
        "optimizer": optimizer.state_dict(),
        "config": args,
        "epoch": cur_epoch,
    }
    path = f'{args.output_dir}/{args.dataset}/model_name_{args.model_name}_llm_model_name_{args.llm_model_name}_llm_frozen_{args.llm_frozen}_max_txt_len_{args.max_txt_len}_max_new_tokens_{args.max_new_tokens}_gnn_model_name_{args.gnn_model_name}_patience_{args.patience}_num_epochs_{args.num_epochs}_seed{args.seed}_checkpoint_{"best" if is_best else cur_epoch}.pth'
    print("Saving checkpoint at epoch {} to {}.".format(cur_epoch, path))
    torch.save(save_obj, path)


def _reload_best_model(model, args):
    """
    Load the best checkpoint for evaluation.
    """
    checkpoint_path = f'{args.output_dir}/{args.dataset}/model_name_{args.model_name}_llm_model_name_{args.llm_model_name}_llm_frozen_{args.llm_frozen}_max_txt_len_{args.max_txt_len}_max_new_tokens_{args.max_new_tokens}_gnn_model_name_{args.gnn_model_name}_patience_{args.patience}_num_epochs_{args.num_epochs}_seed{args.seed}_checkpoint_best.pth'

    print("Loading checkpoint from {}.".format(checkpoint_path))

    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    model.load_state_dict(checkpoint["model"], strict=False)

    return model


def _reload_model(model, checkpoint_path):
    """
    Load the best checkpoint for evaluation.
    """

    print("Loading checkpoint from {}.".format(checkpoint_path))

    checkpoint = torch.load(checkpoint_path, map_location="cpu")
    model.load_state_dict(checkpoint["model"], strict=False)

    return model

In [7]:
from torch_geometric.data import Batch


def collate_fn(original_batch):
    batch = {}
    for k in original_batch[0].keys():
        batch[k] = [d[k] for d in original_batch]
    if 'graph' in batch:
        batch['graph'] = Batch.from_data_list(batch['graph'])
    return batch

In [8]:
import math


def adjust_learning_rate(param_group, LR, epoch, args):
    """Decay the learning rate with half-cycle cosine after warmup"""
    min_lr = 5e-6
    if epoch < args.warmup_epochs: #In epoch 1, the learning rate starts at 0 and gradually increases to LR over the warmup_epochs.
        lr = LR * epoch / args.warmup_epochs
    else:
      #After warmup_epochs, the learning rate decreases to min_lr following a half-cycle cosine schedule.
        lr = min_lr + (LR - min_lr) * 0.5 * (1.0 + math.cos(math.pi * (epoch - args.warmup_epochs) / (args.num_epochs - args.warmup_epochs)))
    param_group["lr"] = lr
    return lr

In [ ]:
args.llm_model_path = llama_model_path[args.llm_model_name]

model = load_model[args.model_name](graph_type=dataset.graph_type, args=args, init_prompt=dataset.prompt)

Loading LLAMA


TypeError: expected str, bytes or os.PathLike object, not NoneType

In [9]:
import argparse


def parse_args_llama():
    parser = argparse.ArgumentParser(description="G-Retriever")

    parser.add_argument("--model_name", type=str, default='graph_llm') #graph_llm
    parser.add_argument("--project", type=str, default="project_g_retriever")
    parser.add_argument("--seed", type=int, default=0)

    parser.add_argument("--dataset", type=str, default='expla_graphs')
    parser.add_argument("--lr", type=float, default=1e-5)
    parser.add_argument("--wd", type=float, default=0.05)
    parser.add_argument("--patience", type=float, default=2)

    # Model Training
    parser.add_argument("--batch_size", type=int, default=1)#default=8
    parser.add_argument("--grad_steps", type=int, default=2)

    # Learning Rate Scheduler
    parser.add_argument("--num_epochs", type=int, default=2) #10
    parser.add_argument("--warmup_epochs", type=float, default=1)

    # Inference
    parser.add_argument("--eval_batch_size", type=int, default=16)

    # LLM related
    parser.add_argument("--llm_model_name", type=str, default='7b')
    parser.add_argument("--llm_model_path", type=str, default='')
    parser.add_argument("--llm_frozen", type=str, default='False')
    parser.add_argument("--llm_num_virtual_tokens", type=int, default=10)
    parser.add_argument("--output_dir", type=str, default='output')
    parser.add_argument("--max_txt_len", type=int, default=128)#default=512
    parser.add_argument("--max_new_tokens", type=int, default=32)

    # GNN related
    parser.add_argument("--gnn_model_name", type=str, default='gt')
    parser.add_argument("--gnn_num_layers", type=int, default=4)
    parser.add_argument("--gnn_in_dim", type=int, default=1024)
    parser.add_argument("--gnn_hidden_dim", type=int, default=1024)
    parser.add_argument("--gnn_num_heads", type=int, default=4)
    parser.add_argument("--gnn_dropout", type=float, default=0.0)

    # args, unknown = parser.parse_known_args()
    args, unknown = parser.parse_known_args()

    return args



In [ ]:


import os
import wandb
import gc
from tqdm import tqdm
import torch
import json
import pandas as pd
from torch.utils.data import DataLoader
from torch.nn.utils import clip_grad_norm_
from transformers.utils import logging

# فعال‌سازی لاگ برای نمایش جزئیات دانلود
logging.set_verbosity_info()  

# from src.model import load_model, llama_model_path
# from src.dataset import load_dataset
# from src.utils.evaluate import eval_funcs
# from src.config import parse_args_llama
# from src.utils.ckpt import _save_checkpoint, _reload_best_model
# from src.utils.collate import collate_fn
# from src.utils.seed import seed_everything
# from src.utils.lr_schedule import adjust_learning_rate


def main(args):

    # Step 1: Set up wandb
    seed = args.seed
    print(f"Initializing training for project: {args.project}")
    
    

    seed_everything(seed=args.seed)
    print(args)

    dataset = load_dataset[args.dataset]() #load Explanation_graph
    idx_split = dataset.get_idx_split() #train-val-test index

    # Step 2: Build Node Classification Dataset
    train_dataset = [dataset[i] for i in idx_split['train']] #contains label, desc, question, graph...
    val_dataset = [dataset[i] for i in idx_split['val']]
    test_dataset = [dataset[i] for i in idx_split['test']]

    train_loader = DataLoader(train_dataset, batch_size=args.batch_size, drop_last=True, pin_memory=True, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_dataset, batch_size=args.batch_size, drop_last=False, pin_memory=True, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_dataset, batch_size=args.eval_batch_size, drop_last=False, pin_memory=True, shuffle=False, collate_fn=collate_fn)

    # Step 3: Build Model
    args.llm_model_path = llama_model_path[args.llm_model_name] #edumunozsala/llama-2-7b-int4-python-code-20k model

    model = load_model[args.model_name](graph_type=dataset.graph_type, args=args, init_prompt=dataset.prompt)#load graph_llm

    # Step 4 Set Optimizer
    params = [p for _, p in model.named_parameters() if p.requires_grad]
    optimizer = torch.optim.AdamW(
        [{'params': params, 'lr': args.lr, 'weight_decay': args.wd}, ],
        betas=(0.9, 0.95)
    )


    trainable_params, all_param = model.print_trainable_params()
    print(f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}")

    # Step 5. Training
    num_training_steps = args.num_epochs * len(train_loader)
    progress_bar = tqdm(range(num_training_steps))
    best_val_loss = float('inf')

    for epoch in range(args.num_epochs):

        model.train()
        epoch_loss, accum_loss = 0., 0.

        for step, batch in enumerate(train_loader):

            optimizer.zero_grad()
            loss = model(batch)
            loss.backward()

            clip_grad_norm_(optimizer.param_groups[0]['params'], 0.1) # Limits the gradient norm to a maximum of 0.1 to prevent exploding gradients.

            if (step + 1) % args.grad_steps == 0: #This condition ensures that the learning rate adjustment only occurs after a specific number of gradient accumulation steps
                adjust_learning_rate(optimizer.param_groups[0], args.lr, step / len(train_loader) + epoch, args) # step / len(train_loader) + epoch : The learning rate is adjusted smoothly not only at the start of each epoch but continuously during the epoch as well.

            optimizer.step() #update weight
            epoch_loss, accum_loss = epoch_loss + loss.item(), accum_loss + loss.item()


            #Log to wandb
            if (step + 1) % args.grad_steps == 0:
                lr = optimizer.param_groups[0]["lr"]
                print(f"Learning Rate: {lr}")
                print(f"Accumulated Loss: {accum_loss / args.grad_steps}")
                accum_loss = 0.


            progress_bar.update(1)

        print(f"Epoch: {epoch}|{args.num_epochs}: Train Loss (Epoch Mean): {epoch_loss / len(train_loader)}")
        #wandb.log({'Train Loss (Epoch Mean)': epoch_loss / len(train_loader)})


#ُStep6 : evaluation on validation
        val_loss = 0.
        eval_output = []
        model.eval()
        with torch.no_grad():
            for step, batch in enumerate(val_loader):
                loss = model(batch)
                val_loss += loss.item()
            val_loss = val_loss/len(val_loader)
            print(f"Epoch: {epoch}|{args.num_epochs}: Val Loss: {val_loss}")
            #wandb.log({'Val Loss': val_loss})

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            _save_checkpoint(model, optimizer, epoch, args, is_best=True)
            best_epoch = epoch

        print(f'Epoch {epoch} Val Loss {val_loss} Best Val Loss {best_val_loss} Best Epoch {best_epoch}')

        if epoch - best_epoch >= args.patience:
            print(f'Early stop at epoch {epoch}')
            break

    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()

    # Step 7. Evaluating on test with infernece part
    os.makedirs(f'{args.output_dir}/{args.dataset}', exist_ok=True)
    path = f'{args.output_dir}/{args.dataset}/model_name_{args.model_name}_llm_model_name_{args.llm_model_name}_llm_frozen_{args.llm_frozen}_max_txt_len_{args.max_txt_len}_max_new_tokens_{args.max_new_tokens}_gnn_model_name_{args.gnn_model_name}_patience_{args.patience}_num_epochs_{args.num_epochs}_seed{seed}.csv'
    print(f'path: {path}')

    model = _reload_best_model(model, args)
    model.eval()
    progress_bar_test = tqdm(range(len(test_loader)))
    with open(path, "w") as f:
        for step, batch in enumerate(test_loader):
            with torch.no_grad():
                output = model.inference(batch)
                df = pd.DataFrame(output)
                for _, row in df.iterrows():
                    f.write(json.dumps(dict(row)) + "\n")
            progress_bar_test.update(1)

    # Step 6. Post-processing & compute metrics
    acc = eval_funcs[args.dataset](path)
    print(f'Test Acc {acc}')
    #wandb.log({'Test Acc': acc})



if __name__ == "__main__":

    args = parse_args_llama()


    main(args)
    torch.cuda.empty_cache()
    torch.cuda.reset_max_memory_allocated()
    gc.collect()

Initializing training for project: project_g_retriever
Namespace(model_name='graph_llm', project='project_g_retriever', seed=0, dataset='expla_graphs', lr=1e-05, wd=0.05, patience=2, batch_size=1, grad_steps=2, num_epochs=2, warmup_epochs=1, eval_batch_size=16, llm_model_name='7b', llm_model_path='', llm_frozen='False', llm_num_virtual_tokens=10, output_dir='output', max_txt_len=128, max_new_tokens=32, gnn_model_name='gt', gnn_num_layers=4, gnn_in_dim=1024, gnn_hidden_dim=1024, gnn_num_heads=4, gnn_dropout=0.0)
Loading LLAMA


loading file tokenizer.model from cache at None
loading file tokenizer.json from cache at /home/ahmadi/.cache/huggingface/hub/models--edumunozsala--llama-2-7b-int4-python-code-20k/snapshots/831ad12e77f97a6431b8505ef75674a48e013fc1/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/ahmadi/.cache/huggingface/hub/models--edumunozsala--llama-2-7b-int4-python-code-20k/snapshots/831ad12e77f97a6431b8505ef75674a48e013fc1/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/ahmadi/.cache/huggingface/hub/models--edumunozsala--llama-2-7b-int4-python-code-20k/snapshots/831ad12e77f97a6431b8505ef75674a48e013fc1/tokenizer_config.json
loading file chat_template.jinja from cache at None
loading configuration file config.json from cache at /home/ahmadi/.cache/huggingface/hub/models--edumunozsala--llama-2-7b-int4-python-code-20k/snapshots/831ad12e77f97a6431b8505ef75674a48e013fc1/config.json
Model config Llama

In [ ]:
model

NameError: name 'model' is not defined

In [ ]:
for name, param in model.named_parameters():
    print(f"Parameter: {name}, requires_grad: {param.requires_grad}")


NameError: name 'model' is not defined

In [ ]:
load_model["llm"]

__main__.LLM

In [ ]:
arg = parse_args_llama()
arg
# model = LLM(args=args,graph_type="Explanation Graph", init_prompt='Question: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of \'support\' or \'counter\'.\n\nAnswer:')
# model


Namespace(model_name='llm', project='project_g_retriever', seed=0, dataset='expla_graphs', lr=1e-05, wd=0.05, patience=2, batch_size=1, grad_steps=2, num_epochs=2, warmup_epochs=1, eval_batch_size=16, llm_model_name='7b', llm_model_path='', llm_frozen='True', llm_num_virtual_tokens=10, output_dir='output', max_txt_len=128, max_new_tokens=32, gnn_model_name='gt', gnn_num_layers=4, gnn_in_dim=1024, gnn_hidden_dim=1024, gnn_num_heads=4, gnn_dropout=0.0)

5f199c000377bebd4c47487f8ac24a26b19ff9cd

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=args.batch_size, drop_last=True, pin_memory=True, shuffle=True, collate_fn=collate_fn)
type(train_loader)

NameError: name 'train_dataset' is not defined

In [55]:
load_dataset = {
    'expla_graphs': ExplaGraphsDataset,
    # 'scene_graphs': SceneGraphsDataset,
    # 'scene_graphs_baseline': SceneGraphsBaselineDataset,
    # 'webqsp': WebQSPDataset,
    # 'webqsp_baseline': WebQSPBaselineDataset,
}
# args = parse_args_llama()
# dataset = load_dataset[args.dataset]()
idx_split = dataset.get_idx_split()

In [ ]:
idx_split

{'train': [96,
  117,
  147,
  180,
  223,
  74,
  127,
  29,
  224,
  201,
  243,
  148,
  265,
  5,
  116,
  56,
  157,
  182,
  65,
  164,
  85,
  179,
  12,
  35,
  28,
  210,
  139,
  51,
  95,
  217,
  186,
  41,
  227,
  183,
  136,
  26,
  275,
  141,
  234,
  0,
  240,
  100,
  216,
  211,
  98,
  36,
  61,
  150,
  196,
  168,
  11,
  273,
  237,
  27,
  185,
  4,
  122,
  32,
  170,
  162,
  175,
  268,
  138,
  62,
  135,
  128,
  267,
  239,
  70,
  231,
  64,
  44,
  195,
  156,
  40,
  123,
  221,
  153,
  23,
  226,
  222,
  81,
  39,
  199,
  249,
  47,
  94,
  229,
  245,
  161,
  43,
  145,
  190,
  250,
  3,
  105,
  53,
  133,
  1,
  232,
  103,
  49,
  163,
  80,
  205,
  34,
  246,
  7,
  171,
  198,
  251,
  110,
  91,
  83,
  129,
  52,
  89,
  8,
  13,
  59,
  88,
  131,
  17,
  166,
  72,
  271,
  134,
  244,
  228,
  264,
  63,
  54,
  107,
  50,
  247,
  174,
  248,
  189,
  14,
  207,
  187,
  219,
  169,
  58,
  218,
  48,
  235,
  252,
  21,
  160,
  191

In [ ]:
len(idx_split["train"])

166

In [ ]:
idx_split["train"]

[96,
 117,
 147,
 180,
 223,
 74,
 127,
 29,
 224,
 201,
 243,
 148,
 265,
 5,
 116,
 56,
 157,
 182,
 65,
 164,
 85,
 179,
 12,
 35,
 28,
 210,
 139,
 51,
 95,
 217,
 186,
 41,
 227,
 183,
 136,
 26,
 275,
 141,
 234,
 0,
 240,
 100,
 216,
 211,
 98,
 36,
 61,
 150,
 196,
 168,
 11,
 273,
 237,
 27,
 185,
 4,
 122,
 32,
 170,
 162,
 175,
 268,
 138,
 62,
 135,
 128,
 267,
 239,
 70,
 231,
 64,
 44,
 195,
 156,
 40,
 123,
 221,
 153,
 23,
 226,
 222,
 81,
 39,
 199,
 249,
 47,
 94,
 229,
 245,
 161,
 43,
 145,
 190,
 250,
 3,
 105,
 53,
 133,
 1,
 232,
 103,
 49,
 163,
 80,
 205,
 34,
 246,
 7,
 171,
 198,
 251,
 110,
 91,
 83,
 129,
 52,
 89,
 8,
 13,
 59,
 88,
 131,
 17,
 166,
 72,
 271,
 134,
 244,
 228,
 264,
 63,
 54,
 107,
 50,
 247,
 174,
 248,
 189,
 14,
 207,
 187,
 219,
 169,
 58,
 218,
 48,
 235,
 252,
 21,
 160,
 191,
 257,
 149,
 130,
 151,
 99,
 87,
 214,
 121,
 276,
 20,
 188,
 71,
 106,
 270,
 102]

In [ ]:
 train_dataset = [dataset[i] for i in idx_split['train']]
 train_dataset

<ipython-input-27-75356012b316>:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  graph = torch.load(f'{PATH}/graphs/{index}.pt')
<ipython-input-27-75356012b316>:26: FutureW

[{'id': 96,
  'label': 'support',
  'desc': 'node_id,node_attr\n0,religion\n1,people\n2,allowed to worship\n3,personal choice\n\nsrc,edge_attr,dst\n0,has context,1\n1,desires,2\n2,is a,3\n',
  'graph': Data(x=[5, 1024], edge_index=[2, 4], edge_attr=[4, 1024], num_nodes=5),
  'question': "Argument 1: Everyone should be allowed to worship whatever God they want, and follow whatever religion they want.\nArgument 2: Religion is a personal choice\nQuestion: Do argument 1 and argument 2 support or counter each other? Answer in one word in the form of 'support' or 'counter'.\n\nAnswer:"},
 {'id': 117,
  'label': 'support',
  'desc': 'node_id,node_attr\n0,language\n1,change\n2,adapt\n3,culture\n\nsrc,edge_attr,dst\n0,capable of,1\n1,part of,2\n0,part of,3\n3,desires,1\n',
  'graph': Data(x=[5, 1024], edge_index=[2, 4], edge_attr=[4, 1024], num_nodes=5),
  'question': "Argument 1: Language should adapt.\nArgument 2: As cultures and environments change, language should adapt with it.\nQuestion: 

In [11]:

import contextlib
import torch
import torch.nn as nn
from torch.cuda.amp import autocast as autocast
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch_scatter import scatter
#from src.model.gnn import load_gnn_model
from peft import (
    LoraConfig,
    get_peft_model,
    # prepare_model_for_int8_training,
    prepare_model_for_kbit_training
)


ModuleNotFoundError: No module named 'torch_scatter'

In [12]:
class GraphTransformer(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers, dropout, num_heads=-1):
        super(GraphTransformer, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(TransformerConv(in_channels=in_channels, out_channels=hidden_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout))
        self.bns = torch.nn.ModuleList()
        self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(TransformerConv(in_channels=hidden_channels, out_channels=hidden_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout,))
            self.bns.append(torch.nn.BatchNorm1d(hidden_channels))
        self.convs.append(TransformerConv(in_channels=hidden_channels, out_channels=out_channels//num_heads, heads=num_heads, edge_dim=in_channels, dropout=dropout,))
        self.dropout = dropout

    def reset_parameters(self):
        for conv in self.convs:
            conv.reset_parameters()
        for bn in self.bns:
            bn.reset_parameters()

    def forward(self, x, adj_t, edge_attr):
        for i, conv in enumerate(self.convs[:-1]):
            x = conv(x, edge_index=adj_t, edge_attr=edge_attr)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.convs[-1](x, edge_index=adj_t, edge_attr=edge_attr)
        return x, edge_attr

In [13]:
load_gnn_model = {
    # 'gcn': GCN,
    # 'gat': GAT,
    'gt': GraphTransformer,
}

In [18]:
import contextlib
import torch
import torch.nn as nn
from torch.cuda.amp import autocast as autocast
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch_scatter import scatter
#from src.model.gnn import load_gnn_model
from peft import (
    LoraConfig,
    get_peft_model,
    # prepare_model_for_int8_training,
    prepare_model_for_kbit_training
)

BOS = '<s>[INST]'  #marks the start of an instruction-based input for the language model.
EOS_USER = '[/INST]' #marks the end of the user input or instruction in an instruction-based input format.
EOS = '</s>' #mark the completion of text generation by the model.

IGNORE_INDEX = -100 #mask non-target tokens during loss computation.


class GraphLLM(torch.nn.Module):

    def __init__(
        self,
        args,
        **kwargs
    ):
        super().__init__()
        self.max_txt_len = args.max_txt_len #maximum length of input text sequences
        self.max_new_tokens = args.max_new_tokens #maximum number of tokens the model is allowed to generate during inference.

        print('Loading LLAMA')
        kwargs = {
            #"max_memory": {0: '80GiB', 1: '80GiB'},
            "device_map": "auto", #Automatically maps the model’s layers across available devices for efficient training/inference.
            "revision": "main", #Specifies the model version or branch to use
        }

        self.tokenizer = AutoTokenizer.from_pretrained(args.llm_model_path,  revision=kwargs["revision"])
        self.tokenizer.pad_token_id = 0
        self.tokenizer.padding_side = 'left'

        model = AutoModelForCausalLM.from_pretrained(
            args.llm_model_path,
            torch_dtype=torch.float16, #used for reduce memory
            low_cpu_mem_usage=True, #used for reduce memory
            **kwargs
        )



        ###LORA configuration

        if args.llm_frozen == 'True':
            print("Freezing LLAMA!")
            for name, param in model.named_parameters():
                param.requires_grad = False
        else:
            print("Training LLAMA with LORA!")
            # model = prepare_model_for_int8_training(model)
            model = prepare_model_for_kbit_training(model)#Reduces the memory footprint while retaining sufficient precision for effective fine-tuning.
            lora_r: int = 8
            lora_alpha: int = 16
            lora_dropout: float = 0.05
            lora_target_modules = [ #Specifies which modules to adapt
                "q_proj",
                "v_proj",
            ]
            config = LoraConfig(
                r=lora_r,
                lora_alpha=lora_alpha,
                target_modules=lora_target_modules,
                lora_dropout=lora_dropout,
                bias="none", #the adaptation does not affect bias terms in the target modules.
                task_type="CAUSAL_LM", # setting up LoRA in models designed for text generation
            )
            model = get_peft_model(model, config)

        self.model = model
        print('Finish loading LLAMA!')



        #GNN configuration for generate embedding
        self.graph_encoder = load_gnn_model[args.gnn_model_name](
            in_channels=args.gnn_in_dim,
            out_channels=args.gnn_hidden_dim,
            hidden_channels=args.gnn_hidden_dim,
            num_layers=args.gnn_num_layers,
            dropout=args.gnn_dropout,
            num_heads=args.gnn_num_heads,
        ).to(self.model.device)


        ## Projection layer to maps the GNN's graph embeddings to a size compatible with the LLM's input embeddings.
        self.projector = nn.Sequential(
            nn.Linear(args.gnn_hidden_dim, 2048),
            nn.Sigmoid(),
            nn.Linear(2048, 4096),
        ).to(self.model.device)

        self.word_embedding = self.model.model.get_input_embeddings() #Accesses the pre-trained word embedding layer from the LLM.

    @property # makes methods look and behave like attributes.

    def device(self):
        return list(self.parameters())[0].device

    def maybe_autocast(self, dtype=torch.bfloat16):#reduces memory usage and accelerates computations on GPUs.
        # if on cpu, don't use autocast
        # if on gpu, use autocast with dtype if provided, otherwise use torch.float16
        enable_autocast = self.device != torch.device("cpu")

        if enable_autocast:
            return torch.cuda.amp.autocast(dtype=dtype)
        else:
            return contextlib.nullcontext()

    def encode_graphs(self, samples):
        graphs = samples['graph']
        graphs = graphs.to(self.model.device)
        n_embeds, _ = self.graph_encoder(graphs.x, graphs.edge_index.long(), graphs.edge_attr)

        # mean pooling
        g_embeds = scatter(n_embeds, graphs.batch, dim=0, reduce='mean')

        return g_embeds

    def forward(self, samples):

        # encode description, questions and labels
        questions = self.tokenizer(samples["question"], add_special_tokens=False)
        descriptions = self.tokenizer(samples["desc"], add_special_tokens=False)
        labels = self.tokenizer(samples["label"], add_special_tokens=False)

        # encode special tokens
        eos_tokens = self.tokenizer(EOS, add_special_tokens=False)
        eos_user_tokens = self.tokenizer(EOS_USER, add_special_tokens=False)
        bos_embeds = self.word_embedding(self.tokenizer(BOS, add_special_tokens=False, return_tensors='pt').input_ids[0])
        pad_embeds = self.word_embedding(torch.tensor(self.tokenizer.pad_token_id)).unsqueeze(0)

        # encode graphs
        graph_embeds = self.encode_graphs(samples)
        graph_embeds = self.projector(graph_embeds)

        batch_size = len(samples['id'])
        batch_inputs_embeds = []
        batch_attention_mask = []
        batch_label_input_ids = []
        for i in range(batch_size):
            # Add bos & eos token
            label_input_ids = labels.input_ids[i][:self.max_new_tokens] + eos_tokens.input_ids
            input_ids = descriptions.input_ids[i][:self.max_txt_len] + questions.input_ids[i] + eos_user_tokens.input_ids + label_input_ids
            inputs_embeds = self.word_embedding(torch.tensor(input_ids).to(self.model.device))
            inputs_embeds = torch.cat([bos_embeds, graph_embeds[i].unsqueeze(0), inputs_embeds], dim=0)

            batch_inputs_embeds.append(inputs_embeds)
            batch_attention_mask.append([1] * inputs_embeds.shape[0])
            label_input_ids = [IGNORE_INDEX] * (inputs_embeds.shape[0]-len(label_input_ids))+label_input_ids
            batch_label_input_ids.append(label_input_ids)

        # pad inputs_embeds
        max_length = max([x.shape[0] for x in batch_inputs_embeds])
        for i in range(batch_size):
            pad_length = max_length-batch_inputs_embeds[i].shape[0]
            batch_inputs_embeds[i] = torch.cat([pad_embeds.repeat(pad_length, 1), batch_inputs_embeds[i]])
            batch_attention_mask[i] = [0]*pad_length+batch_attention_mask[i]
            batch_label_input_ids[i] = [IGNORE_INDEX] * pad_length+batch_label_input_ids[i]

        inputs_embeds = torch.stack(batch_inputs_embeds, dim=0).to(self.model.device)
        attention_mask = torch.tensor(batch_attention_mask).to(self.model.device)
        label_input_ids = torch.tensor(batch_label_input_ids).to(self.model.device)

        with self.maybe_autocast():
            outputs = self.model(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                return_dict=True,
                labels=label_input_ids,
            )

        return outputs.loss

    def inference(self, samples):

        # encode description and questions
        questions = self.tokenizer(samples["question"], add_special_tokens=False)
        descriptions = self.tokenizer(samples["desc"], add_special_tokens=False)

        # encode special tokens
        eos_user_tokens = self.tokenizer(EOS_USER, add_special_tokens=False)
        bos_embeds = self.word_embedding(self.tokenizer(BOS, add_special_tokens=False, return_tensors='pt').input_ids[0])
        pad_embeds = self.word_embedding(torch.tensor(self.tokenizer.pad_token_id)).unsqueeze(0)

        # encode graphs
        graph_embeds = self.encode_graphs(samples)
        graph_embeds = self.projector(graph_embeds)

        batch_size = len(samples['id'])
        batch_inputs_embeds = []
        batch_attention_mask = []
        for i in range(batch_size):
            # Add bos & eos token
            input_ids = descriptions.input_ids[i][:self.max_txt_len] + questions.input_ids[i] + eos_user_tokens.input_ids
            inputs_embeds = self.word_embedding(torch.tensor(input_ids).to(self.model.device))
            inputs_embeds = torch.cat([bos_embeds, graph_embeds[i].unsqueeze(0), inputs_embeds], dim=0)
            batch_inputs_embeds.append(inputs_embeds)
            batch_attention_mask.append([1] * inputs_embeds.shape[0])

        # pad inputs_embeds
        max_length = max([x.shape[0] for x in batch_inputs_embeds])
        for i in range(batch_size):
            pad_length = max_length-batch_inputs_embeds[i].shape[0]
            batch_inputs_embeds[i] = torch.cat([pad_embeds.repeat(pad_length, 1), batch_inputs_embeds[i]])
            batch_attention_mask[i] = [0]*pad_length+batch_attention_mask[i]

        inputs_embeds = torch.stack(batch_inputs_embeds, dim=0).to(self.model.device)
        attention_mask = torch.tensor(batch_attention_mask).to(self.model.device)

        with self.maybe_autocast():
            outputs = self.model.generate(
                inputs_embeds=inputs_embeds,
                max_new_tokens=self.max_new_tokens,
                attention_mask=attention_mask,
                # do_sample=True,
                use_cache=True  # IMPORTANT!
            )
        pred = self.tokenizer.batch_decode(outputs, skip_special_tokens=True)

        return {'id': samples['id'],
                'pred': pred,
                'label': samples['label'],
                'question': samples['question'],
                'desc': samples['desc'], }

    def print_trainable_params(self):
        trainable_params = 0
        all_param = 0

        for _, param in self.named_parameters():
            num_params = param.numel() #the total number of elements in the parameter tensor.

            all_param += num_params
            if param.requires_grad:
                trainable_params += num_params

        return trainable_params, all_param


    #exmp:
#     import torch.nn as nn

# class SimpleModel(nn.Module):
#     def __init__(self):
#         super(SimpleModel, self).__init__()
#         self.fc1 = nn.Linear(10, 20)
#         self.fc2 = nn.Linear(20, 5)

#     def forward(self, x):
#         return self.fc2(self.fc1(x))

# model = SimpleModel()
#Trainable Parameters: 265, Total Parameters: 265



/home/ahmadi/sadaf/GraphNeighborLM/sadafenv2/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/home/ahmadi/sadaf/GraphNeighborLM/sadafenv2/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feed

In [ ]:
args.llm_model_path

'edumunozsala/llama-2-7b-int4-python-code-20k'

In [ ]:
AutoTokenizer.from_pretrained("edumunozsala/llama-2-7b-int4-python-code-20k", )

TypeError: expected str, bytes or os.PathLike object, not NoneType

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("edumunozsala/llama-2-7b-int4-python-code-20k")
tokenizer

LlamaTokenizerFast(name_or_path='edumunozsala/llama-2-7b-int4-python-code-20k', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	32000: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=False),
}

In [ ]:
!nvidia-smi

Thu Sep  5 08:59:32 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0              28W /  70W |  12053MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--